## Quantile regression

First, import from pickle and packages

In [ ]:
import pickle
import os
import pandas as pd
import numpy as np
from scipy.stats import johnsonsu, uniform
from sklearn.linear_model import QuantileRegressor

path = os.getcwdb().decode("utf-8")+"/data"

In [ ]:
scaler_x, X_scaled, y_a1, y_a2, y_sp, X_test_scaled, y_test_a1, y_test_a2, y_test_sp, feature_names, imp_order = pickle.load(open(path+"/pickle_1.pkl", "rb"))

Prepare data for shifting window regression

In [ ]:
X_scaled=np.concatenate([X_scaled, X_test_scaled])
y_a1=np.concatenate([y_a1, y_test_a1])
y_a2=np.concatenate([y_a2, y_test_a2])
y_sp=np.concatenate([y_sp, y_test_sp])

In [ ]:
test_len = len(y_test_a1)
quantiles = np.linspace(start=0.01, stop=0.99, num=99)
predictions_a1 = np.zeros([len(X_test_scaled), len(quantiles)])
predictions_a2 = np.zeros([len(X_test_scaled), len(quantiles)])
predictions_sp = np.zeros([len(X_test_scaled), len(quantiles)])
row_ind = 0
fit_res_a1=[]
fit_res_a2=[]
fit_res_sp=[]

Run quantile regression with shifts of 720 data points. Includes failsafe if not converging (did not happen in practice).

In [ ]:
while test_len>0:
    ind = 0
    for quantile in quantiles:
        qr1 = QuantileRegressor(quantile=quantile, alpha=0.01, solver="highs")
        qr2 = QuantileRegressor(quantile=quantile, alpha=0.01, solver="highs")
        qr3 = QuantileRegressor(quantile=quantile, alpha=0.01, solver="highs")
        try:    
            if test_len>=720:
                y_pred_a1 = qr1.fit(X_scaled[:-test_len,:], y_a1[:-test_len].ravel()).predict(X_scaled[-test_len:-test_len+24*30,:])
            else:
                y_pred_a1 = qr1.fit(X_scaled[:-test_len,:], y_a1[:-test_len].ravel()).predict(X_scaled[-test_len:,:])
        except:
            y_pred_a1 = [-1000000000 for i in range(min(720, test_len))]
        try:    
            if test_len>=720:
                y_pred_a2 = qr2.fit(X_scaled[:-test_len,:], y_a2[:-test_len].ravel()).predict(X_scaled[-test_len:-test_len+24*30,:])
            else:
                y_pred_a2 = qr2.fit(X_scaled[:-test_len,:], y_a2[:-test_len].ravel()).predict(X_scaled[-test_len:,:])
        except:
            y_pred_a2 = [-1000000000 for i in range(min(720, test_len))]
        try:
            if test_len>=720:
                y_pred_sp = qr3.fit(X_scaled[:-test_len,:], y_sp[:-test_len].ravel()).predict(X_scaled[-test_len:-test_len+24*30,:])
            else:
                y_pred_sp = qr3.fit(X_scaled[:-test_len,:], y_sp[:-test_len].ravel()).predict(X_scaled[-test_len:,:])
        except:
            y_pred_sp = [-1000000000 for i in range(min(720, test_len))]  
        predictions_a1[row_ind:min(row_ind+24*30,len(predictions_a1)),ind] = y_pred_a1
        predictions_a2[row_ind:min(row_ind+24*30,len(predictions_a2)),ind] = y_pred_a2
        predictions_sp[row_ind:min(row_ind+24*30,len(predictions_sp)),ind] = y_pred_sp
        ind+=1
    test_len-=24*30
    row_ind+=24*30
    pickle_obj=[fit_res_a1, fit_res_a2, fit_res_sp, predictions_a1, predictions_a2, predictions_sp]
    pickle.dump(pickle_obj, open(path+"/pickle_qr.pkl", "wb"))


In [ ]:
from sklearn.metrics import mean_pinball_loss
def pinball_score_qr(y, y_pred):
    loss_total = 0
    loss_list = []
    for i in range(1,100):
        #y_pred = np.zeros(len(y))
        #for j in range(len(y)):
        #    y_pred[j] = johnsonsu.ppf(i/100, params[j,0], params[j,1], params[j,2], params[j,3])
        #print(max(y_pred))    
        loss = mean_pinball_loss(y, y_pred[:,i-1], alpha=i/100)
        loss_list.append(loss)
        #print(loss)
        loss_total+=loss
    return loss_total/99, loss_list

Fit quantiles to Johnson's SU distribution

In [ ]:
fit_res_a1=[]
fit_res_a2=[]
fit_res_sp=[]
for i in range(len(X_test_scaled)):
    fit_res_a1.append(johnsonsu.fit(predictions_a1[i,:], fscale=1))    
    fit_res_a2.append(johnsonsu.fit(predictions_a2[i,:], fscale=1))    
    fit_res_sp.append(johnsonsu.fit(predictions_sp[i,:], fscale=1))

Save to pickle

In [ ]:
pickle_obj=[fit_res_a1, fit_res_a2, fit_res_sp, predictions_a1, predictions_a2, predictions_sp]
pickle.dump(pickle_obj, open(path+"/pickle_qr_005.pkl", "wb"))
